# Dataset Exploration

Let's run a few queries against our database to attempt to understand what information we have.  First, we need to set up a few contextual variables.

In [39]:
import pandas as pd
from py2neo import Graph

# Connect to our Graph database, ensure connectivity, and store connection in variable.
graph = Graph("bolt://localhost", auth=("neo4j", "neo4j"));

Now, we can run a simple query to give us our first look into the dataset.  The below query will give us all of our different node types and how many of those types we have.

In [5]:
result = {"label": [], "count": []}
for label in graph.run("CALL db.labels()").to_series():
    query = f"MATCH (:`{label}`) RETURN count(*) as count"
    count = graph.run(query).to_data_frame().iloc[0]['count']
    result["label"].append(label)
    result["count"].append(count)
nodes_df = pd.DataFrame(data=result)
nodes_df.sort_values("count")

,label,count
0,Scan,2
2,Hit,9
1,Hop,61


The table above shows we have two scans that resulted in 9 'Hits' and 61 'Hops'.  At first glance, we may be quick to assume 9 'Hits' means we ended up at nine different sites.  What about a redirect scenario?  How can we detect if a 'Hit' resulted in a redirect or not?  Below is a list containing all of the 'Hits' we encountered:

In [16]:
query = """
MATCH (h:Hit)
RETURN h.url as URL, h.ip AS IP
"""

graph.run(query).to_data_frame()

,IP,URL
0,77.23.22.15,http://www.prezbolak2016.com/
1,77.23.45.22,http://www.prezbarak2016.com/
2,77.23.22.01,http://www.prezobama2016.com/
3,230.24.41.120,http://www.notmyobama.com/
4,112.43.82.28,http://www.thumpforpresident.com/
5,77.23.45.16,http://www.preztrump2016.com/
6,25.231.67.78,http://www.superdonald2016.com/
7,145.34.33.120,http://www.thedonld4prez.com/
8,145.34.33.130,http://www.notmytrump.com/


Knowing we may have some 'Hits' that resulted in redirects, it would be nice to know which ones were the result of a redirect, but did not redirect themselves.

In [26]:
query = """
MATCH p=(h:Hit)-[r:RESULTED_IN*]->(hh:Hit)
WITH DISTINCT last(nodes(p)) as nodes
RETURN nodes.url as URL, nodes.ip AS IP
"""

graph.run(query).to_data_frame()

,IP,URL
0,77.23.45.22,http://www.prezbarak2016.com/
1,145.34.33.120,http://www.thedonld4prez.com/
2,145.34.33.130,http://www.notmytrump.com/


While knowing what websites were the end of a series of 'Hops', we could also find out which 'Hits' result in  redirects.

In [24]:
query = """
MATCH (:Scan)-[r:RESULTED_IN*]->(h:Hit)-[rr:RESULTED_IN*]->(hh:Hit)
WITH DISTINCT h
RETURN h.url as URL, h.ip AS IP
"""

graph.run(query).to_data_frame()

,IP,URL
0,77.23.22.15,http://www.prezbolak2016.com/
1,77.23.22.01,http://www.prezobama2016.com/
2,25.231.67.78,http://www.superdonald2016.com/


But what about the other three 'Hits'?  Why weren't they returned in the last two queries?  Simple: the other 'Hits' are websites that did not redirect or result from a redirect themselves.

In [32]:
query = """
MATCH p=(s:Scan)-[:RESULTED_IN*1..]->(h:Hit)
WHERE NOT (h)-[]->() AND NOT (:Hit)-[*]->(h)
WITH DISTINCT h
RETURN h.url as URL, h.ip AS IP
"""

graph.run(query).to_data_frame()

,IP,URL
0,230.24.41.120,http://www.notmyobama.com/
1,112.43.82.28,http://www.thumpforpresident.com/
2,77.23.45.16,http://www.preztrump2016.com/


What is we simply wanted to know the number of tests that were run for a single scan?

In [33]:
query = """
MATCH (s:Scan { id: '1' })-[r:RESULTED_IN]->()
RETURN size(collect(r)) as Test_Count
"""

graph.run(query).to_data_frame()

,Test_Count
0,4


Let's try to look at more intereting examples.  What if we wanted to look into the longest path a test took from start to its final 'Hit'?  We could easily write a simple query to find this information out.

In [36]:
query = """
match p=(n:Scan)-[:RESULTED_IN*1..]->(m:Hit)
WITH relationships(p) as n, length(p) as Length
RETURN n[0].test_id as Test_ID, Length
order by Length desc
limit 3
"""

graph.run(query).to_data_frame()

,Length,Test_ID
0,15,d7de307f-1ecf-4d93-9271-1d318b79e883
1,13,ad3bc684-0cb5-4860-8afa-77a6dbd3e87c
2,10,effea848-5f94-4adf-ba99-01ce37960570


Another interesting question we could ask is: What 'Hop' and 'Hit' carry the most relationships?  The following query will let us know what ip addresses and websites are the most connected.

In [38]:
query = """
MATCH (n)
WHERE n:Hit OR n:Hop
WITH n, (SIZE((n)-[]->()) + SIZE(()-[]->(n))) AS Relationship_Count
RETURN n.ip, n.url, Relationship_Count
ORDER By Relationship_Count DESC
LIMIT 5
"""

graph.run(query).to_data_frame()

,Relationship_Count,n.ip,n.url
0,4,14.195.80.207,None
1,3,77.23.45.22,http://www.prezbarak2016.com/
2,2,77.23.22.01,http://www.prezobama2016.com/
3,2,77.23.22.15,http://www.prezbolak2016.com/
4,2,100.204.179.91,None
